In [1]:
import numpy as np

In [2]:
transition_file = 'transitionDists.json'
emission_file = 'emissionDists.json'

In [3]:
import json
with open(transition_file) as f:
    transitionDists = json.load(f)
with open(emission_file) as f:
    emissionDists = json.load(f)

In [4]:
states = list(transitionDists.keys())
end_tag_dict = {}
for st in states:
    end_tag_dict[st] = -10000000000
end_tag_dict['<DUMMY_END_TAG>'] = -10000000000
transitionDists['<DUMMY_END_TAG>'] = end_tag_dict

In [5]:
def print_dict(d):
    for key in d:
        print(key)
        print(d.get(key))
        print("----")

In [6]:
print_dict(transitionDists)

<DUMMY_START_TAG>
{'PROPN': -2.1467083945212253, 'PUNCT': -3.366100839244863, 'NUM': -3.4304700527731917, 'DET': -2.2989170284577893, 'PRON': -1.2671385359498315, 'SCONJ': -3.3387705114436352, 'ADP': -3.154615842148185, 'NOUN': -2.7661764319384545, 'CCONJ': -3.7668437963380854, 'ADV': -2.531216015772903, 'ADJ': -3.250655123414034, 'VERB': -2.795756777942669, 'X': -3.9435966061433665, 'AUX': -3.561857897181712, 'INTJ': -3.4503662574591725, 'SYM': -4.861328112168165, 'PART': -5.673546490355742, '<DUMMY_START_TAG>': -11.73965458045949, '<DUMMY_END_TAG>': -11.73965458045949}
----
PROPN
{'PUNCT': -1.4141345165822958, 'PROPN': -1.322530396703237, 'ADP': -2.777722398509882, 'AUX': -2.805428490058925, 'ADV': -4.088330814931571, 'NOUN': -2.5678575279065043, 'CCONJ': -2.8118338411929327, 'VERB': -2.7789652522617136, 'SCONJ': -5.176860525060084, 'NUM': -3.47447811903981, 'PART': -3.1677196277455817, 'DET': -5.356177025638267, 'PRON': -4.631613648844943, 'ADJ': -4.8137766139329115, '<DUMMY_END_TAG

In [7]:
def inverse_emission(emissionDists,transitionDists):
    states = list(transitionDists.keys())
    ## fid all the unique observation tokens
    new_dict = {}
    for tag_key in emissionDists:
        for token_key in emissionDists.get(tag_key):
            if(new_dict.get(token_key) == None):
                new_dict[token_key] = {}
            new_dict[token_key][tag_key] = emissionDists.get(tag_key).get(token_key)
            
    for token_key in new_dict:
        emission_prob_dict = new_dict.get(token_key)
        for st in states:
            if(emission_prob_dict.get(st) == None):
                emission_prob_dict[st] = -10000000000
            
    return new_dict

In [8]:
def get_emission_vector(transitionDists,inverese_emission_dict,token):
    states = list(transitionDists.keys())
    #print(states)
    emission_dict = inverese_emission_dict.get(token)
    emission_list = []
    for st in states:
        emission_list.append(emission_dict.get(st))
    return np.array(emission_list)

In [42]:
def get_transition_vector(transitionDists,tag,flag):
    
    states = list(transitionDists.keys())
    
    tag_index= None
    for i,st in enumerate(states):
        if(st == tag):
            tag_index = i
                        
    transition_matrix = []
    for st in states:
        trans = transitionDists.get(st)
        transition_list = []
        for s in states:
            transition_list.append(trans.get(s))
        transition_matrix.append(np.array(transition_list))
    
    transition_matrix = np.array(transition_matrix)
        
    if(flag == 'curr_fixed'):
        return transition_matrix[tag_index]
    elif(flag == 'curr_not_fixed'):
        return transition_matrix[:,tag_index]

In [10]:
'''def get_transition_vector(transitionDists,tag):
    states = list(transitionDists.keys()) + ['<DUMMY_END_TAG>']
    #print(states)
    trans = transitionDists.get(tag)
    transition_list = []
    for st in states:
        transition_list.append(trans.get(st))
    return np.array(transition_list)'''

"def get_transition_vector(transitionDists,tag):\n    states = list(transitionDists.keys()) + ['<DUMMY_END_TAG>']\n    #print(states)\n    trans = transitionDists.get(tag)\n    transition_list = []\n    for st in states:\n        transition_list.append(trans.get(st))\n    return np.array(transition_list)"

In [11]:
inverese_emission_dict = inverse_emission(emissionDists,transitionDists)
print(len(inverese_emission_dict))
print(inverese_emission_dict.get('Professor'))
print(len(inverese_emission_dict.get('Professor')))

19673
{'PROPN': -8.369546048853167, 'NOUN': -10.384374920258649, '<DUMMY_START_TAG>': -10000000000, 'PUNCT': -10000000000, 'ADJ': -10000000000, 'VERB': -10000000000, 'DET': -10000000000, 'ADP': -10000000000, 'AUX': -10000000000, 'PRON': -10000000000, 'PART': -10000000000, 'SCONJ': -10000000000, 'NUM': -10000000000, 'ADV': -10000000000, 'CCONJ': -10000000000, 'X': -10000000000, 'INTJ': -10000000000, 'SYM': -10000000000, '<DUMMY_END_TAG>': -10000000000}
19


In [120]:
inverese_emission_dict.get('stupid')

In [12]:
emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,'Professor')
print(emission_vector)

[-1.00000000e+10 -8.36954605e+00 -1.00000000e+10 -1.00000000e+10
 -1.03843749e+01 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10]


In [13]:
transition_vector = get_transition_vector(transitionDists,'ADJ','curr_fixed')
print(transition_vector)

[-11.73453985  -3.59102244  -1.9946895   -2.95636797  -0.60755631
  -4.66892648  -5.31944289  -2.53819699  -5.66843176  -4.29556826
  -3.4278208   -3.69184035  -4.90907981  -4.31336232  -3.1074166
  -6.9387493   -8.30055264  -6.71726001  -5.51793375]


In [121]:
observation = ['stupid','Professor','is','refusing','to','cooperate']

In [15]:
start_prob_vector = []
for i in  range(len(transitionDists)):
    start_prob_vector.append(np.log(1/(len(transitionDists))))
start_prob_vector = np.array(start_prob_vector)
print(start_prob_vector)

[-2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898 -2.94443898
 -2.94443898]


In [16]:
'''inverese_emission_dict = {}
transitionDists = {}
inverese_emission_dict[1] = np.log(np.array([0.7,0.4,0.8]))
inverese_emission_dict[2] = np.log(np.array([0.3,0.6,0.2]))
transitionDists[1] = np.log(np.array([0.5,0.3,0.2]))
transitionDists[2] = np.log(np.array([0.0000000000000001,0.6,0.4]))
transitionDists[3] = np.log(np.array([0.0000000000000001,0.0000000000000001,1.0]))'''

'inverese_emission_dict = {}\ntransitionDists = {}\ninverese_emission_dict[1] = np.log(np.array([0.7,0.4,0.8]))\ninverese_emission_dict[2] = np.log(np.array([0.3,0.6,0.2]))\ntransitionDists[1] = np.log(np.array([0.5,0.3,0.2]))\ntransitionDists[2] = np.log(np.array([0.0000000000000001,0.6,0.4]))\ntransitionDists[3] = np.log(np.array([0.0000000000000001,0.0000000000000001,1.0]))'

In [17]:
'''observation = [1,2,1]'''

'observation = [1,2,1]'

In [18]:
'''start_prob_vector = np.log(np.array([0.9,0.1,0.00000001]))'''

'start_prob_vector = np.log(np.array([0.9,0.1,0.00000001]))'

In [19]:
## compute P(h_t|v_1-t)
def forward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    states = transitionDists.keys()
    a_messages = []
    
    for i, obs in enumerate(observation):
        print(obs)
        if(i==0):
            ## log(p(v1|h)) + log(p(h))
            emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
            a_0 = emission_vector + start_prob_vector
            a_messages.append(a_0)
        else:
            a = np.zeros(len(states))
            for j,st in enumerate(states):
                emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
                transition_vector = get_transition_vector(transitionDists,st,'curr_fixed')
                a += np.exp(emission_vector + transition_vector + np.array(a_messages[-1][j]))
            a_messages.append(np.log(a))
            
    return a_messages    

In [20]:
a_messages = forward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector)

Professor
is
refusing
to
cooperate


/home/apostolos/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


In [21]:
def filtering(a_messages,i):
    numerator = np.exp(a_messages[i])
    denominator = np.sum(numerator,axis = 0)
    ## return both the proba distr and the likelihood
    return numerator/denominator,denominator

In [22]:
for i in range(len(a_messages)):
    print(filtering(a_messages,i))

(array([0.        , 0.88234524, 0.        , 0.        , 0.11765476,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ]), 1.382802592133662e-05)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.75985926e-02, 0.00000000e+00, 0.00000000e+00,
       9.22125467e-01, 7.89913329e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.96949493e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 1.3847315841952405e-07)
(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 5.970722448914806e-12)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.84658169e-05, 1.31187802e-05, 0.00000000e+00, 3.15440433e-01,
       0.00000000e+00, 0.00000000e+00, 6.71576141e-01, 1.21755932e-02,
       0.00000000e+00,

In [23]:
def reverse(text):
    result = []
    for i in range(len(text),0,-1):
        result.append(text[i-1])
    return result

In [24]:
obs = reverse(observation)
print(obs)

['cooperate', 'to', 'refusing', 'is', 'Professor']


In [25]:
def backward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    observation = reverse(observation)

    states = transitionDists.keys()
    b_messages = []
    
    b_0 = np.zeros(len(states))
    b_messages.append(b_0)
    
    for i, obs in enumerate(observation):
        if(i==len(observation)-1):
            break
        b = np.zeros(len(states))
        emission_vector = get_emission_vector(transitionDists,inverese_emission_dict,obs)
        for j,st in enumerate(states):   
            transition_vector = get_transition_vector(transitionDists,st,"curr_not_fixed")
            b += np.exp(emission_vector[j] + transition_vector + np.array(b_messages[-1][j]))
        b_messages.append(np.log(b))
    
    b_messages.reverse()
    
    return b_messages   

In [26]:
b_messages = backward_algo(observation,inverese_emission_dict,transitionDists,start_prob_vector)

/home/apostolos/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


In [27]:
for b in b_messages:
    print(np.exp(b))

[2.62154371e-13 5.53988822e-13 1.39870119e-13 3.19162403e-14
 6.36742582e-13 5.62868734e-14 1.70321142e-14 7.43360634e-15
 7.31533016e-13 2.90828560e-12 6.60924865e-13 1.40216188e-13
 1.07860792e-13 3.96569767e-13 5.16961026e-13 5.53795607e-14
 2.12282016e-13 1.72111074e-14 0.00000000e+00]
[1.05311309e-11 1.07094577e-11 7.80717377e-12 1.61795987e-12
 9.96133376e-12 2.79591173e-12 3.28296886e-12 1.15447867e-12
 6.08016493e-11 4.71529662e-11 1.22412322e-10 2.72431865e-11
 1.90128057e-12 3.32950442e-11 2.99480561e-11 2.45802101e-12
 6.00154088e-11 2.32455362e-12 0.00000000e+00]
[7.21844134e-08 8.12764061e-07 6.89497630e-08 6.30178803e-07
 5.43495312e-07 1.30557176e-06 5.38635703e-09 1.57062533e-08
 2.21808177e-06 2.30278112e-07 2.86285008e-08 1.23911340e-07
 4.70360989e-08 3.61507184e-07 2.33156380e-07 3.40461101e-09
 3.72647733e-09 3.62560936e-08 0.00000000e+00]
[2.86223479e-06 2.91070186e-06 2.12189598e-06 4.39742042e-07
 2.70737077e-06 7.59895197e-07 8.92271469e-07 3.13773424e-07
 1.65

In [28]:
def smoothing(b_messages,a_messages,i):
    num = np.exp(a_messages[i])*np.exp(b_messages[i])
    denom = np.sum(num,axis = 0)
    return num/denom,denom

In [29]:
for i in range(len(b_messages)):
    print(smoothing(b_messages,a_messages,i))

(array([0.        , 0.86710583, 0.        , 0.        , 0.13289417,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ]), 7.795206617921301e-18)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.85403157e-03, 0.00000000e+00, 0.00000000e+00,
       9.95963318e-01, 6.61647121e-05, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.16485651e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), 7.795206617921312e-18)
(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), 7.795206617921292e-18)
(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       9.42513831e-06, 4.42289522e-07, 0.00000000e+00, 4.39129904e-03,
       0.00000000e+00, 0.00000000e+00, 9.91311475e-01, 3.99979535e-03,
       0.00000000e+00, 

In [122]:
def viterbi(observation,inverese_emission_dict,transitionDists,start_prob_vector):
    
    states = list(transitionDists.keys())
    print(states)
    
    # length of the observation sequence
    T = len(observation)
    # number of states
    K = len(states)
    
    T1 = np.zeros((K,T))
    T2 = np.zeros((K,T))
    
    ## initialise the first code of T1
    for i in range(K):
        T1[i,0] = start_prob_vector[i] + get_emission_vector(transitionDists,inverese_emission_dict,observation[0])[i]

    
    for j in range(1,T):
        for i in range(K):
            proba_array = np.zeros(K)
            # get the transition to state i from all the k previous states
            trans = get_transition_vector(transitionDists,states[i],"curr_not_fixed")
            for k in range(K):
                transition = trans[k]
                # get the emission form the current state to the observation
                emission = get_emission_vector(transitionDists,inverese_emission_dict,observation[j])[i]
                # multiply the emsiion the transition and the proba of being in the previous k-th state
                proba_array[k] = T1[k][j-1] + transition + emission
            
            #print(proba_array)
            #print(np.max(proba_array))
            #print(np.argmax(proba_array))
            #print("----")
            T1[i,j] = np.max(proba_array)
            T2[i,j] = np.argmax(proba_array)
            
    print(T2)
    print(T1)
            
      
    max_states = []
    indices = []
    zt = np.argmax(T1[:,-1])

    max_states.append(states[zt])
    indices.append(zt)
    
    for i in range(T-1,0,-1):
        zt = T2[zt,i]
        zt = int(zt)
        indices.append(zt)
        max_states.append(states[zt])
    
    path = reverse(max_states)
    reversed_indices = reverse(indices)
    
        
    return path, reversed_indices
        
    
        
    

In [123]:
viterbi(observation,inverese_emission_dict,transitionDists,start_prob_vector)

['<DUMMY_START_TAG>', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADP', 'AUX', 'PRON', 'PART', 'SCONJ', 'NUM', 'ADV', 'CCONJ', 'X', 'INTJ', 'SYM', '<DUMMY_END_TAG>']
[[ 0.  3.  1.  8.  5. 10.]
 [ 0.  3.  1.  8.  5.  7.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  5.  5. 10.]
 [ 0.  3.  1.  8.  5.  7.]
 [ 0.  3.  4.  8.  5. 10.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  4.  8.  5.  7.]
 [ 0.  3.  1.  8.  5. 10.]]
[[-1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
  -1.00000000e+10 -1.00000001e+10]
 [-1.00000000e+10 -2.22650251e+01 -1.00000000e+10 -1.00000000e+10
  -1.00000000e+10 -1.00000000e+10]
 [-1.00000000e+10 -1.00000000e+10 -1.00000000e+10 -1.00000000e+10
  -1.00000000e+10 -1.00000000e+10]
 [-1.0304456

(['ADJ', 'NOUN', 'AUX', 'VERB', 'PART', 'VERB'], [3, 4, 8, 5, 10, 5])